In [ ]:
# !pip3 install numpy==1.23.5 -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install jiwer -i https://mirrors.aliyun.com/pypi/simple/

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import chardet
from pymcd.mcd import Calculate_MCD
import scipy.stats as st
import numpy as np
import glob
import soundfile as sf
from tqdm import tqdm
import os
import pandas as pd
import librosa
import torch
from sklearn.preprocessing import StandardScaler

import sys
default_path = sys.path.copy()

sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
# from pytorch import cuda2numpy, cuda2cpu
from basic import plot_spectrogram, select_columns, get_bool_base_on_conditions, plot_confusion_matrix
from sound import play_audio
# import librosa

import whisper
from whisper.normalizers.english import EnglishTextNormalizer
normalizer = EnglishTextNormalizer()
import jiwer


intensities = np.arange(0, 10.1)*0.1
intensities = list(np.round(intensities, 1))[::2]
emos = list(set(["Angry", "Happy", "Neutral", "Sad", "Surprise"])-set(["Neutral"]))
emos.sort()

result_base_dir = "evaluation/"

In [ ]:
models = {
    # "tiny": whisper.load_model("tiny.en"),
    # "base": whisper.load_model("base.en"),
    # "small": whisper.load_model("small.en"),
    # "medium": whisper.load_model("medium.en"),
    # "turbo": whisper.load_model("turbo"),
    "large": whisper.load_model("large"),
}

In [ ]:
dataset = "ESD"
sr = 16000


if dataset=="ESD":
    results = {os.path.basename(p[:-1]): p for p in glob.glob("audio/240529_TAC/expressiveness/*/")}
    depth = 3
    mcd_filenames = ['0011_000354', '0011_000357', '0011_000361', '0011_000367', '0011_000701', '0011_000705', '0011_000710', '0011_000716', '0011_000002', '0011_000003', '0011_000014', '0011_000018', '0011_001054', '0011_001057', '0011_001064', '0011_001069', '0011_001402', '0011_001403', '0011_001411', '0011_001412', '0012_000353', '0012_000356', '0012_000362', '0012_000368', '0012_000702', '0012_000703', '0012_000706', '0012_000720', '0012_000007', '0012_000012', '0012_000016', '0012_000018', '0012_001054', '0012_001056', '0012_001057', '0012_001058', '0012_001404', '0012_001406', '0012_001408', '0012_001419', '0013_000354', '0013_000356', '0013_000357', '0013_000360', '0013_000708', '0013_000710', '0013_000715', '0013_000720', '0013_000002', '0013_000009', '0013_000012', '0013_000017', '0013_001052', '0013_001053', '0013_001056', '0013_001067', '0013_001404', '0013_001410', '0013_001415', '0013_001420', '0014_000354', '0014_000367', '0014_000368', '0014_000369', '0014_000703', '0014_000704', '0014_000714', '0014_000715', '0014_000001', '0014_000004', '0014_000005', '0014_000020', '0014_001051', '0014_001058', '0014_001062', '0014_001067', '0014_001402', '0014_001411', '0014_001414', '0014_001419', '0015_000358', '0015_000361', '0015_000364', '0015_000370', '0015_000702', '0015_000704', '0015_000708', '0015_000711', '0015_000001', '0015_000002', '0015_000003', '0015_000011', '0015_001059', '0015_001061', '0015_001064', '0015_001068', '0015_001408', '0015_001411', '0015_001416', '0015_001419', '0016_000360', '0016_000362', '0016_000364', '0016_000367', '0016_000701', '0016_000708', '0016_000712', '0016_000717', '0016_000005', '0016_000010', '0016_000011', '0016_000020', '0016_001058', '0016_001062', '0016_001067', '0016_001069', '0016_001401', '0016_001411', '0016_001412', '0016_001416', '0017_000351', '0017_000354', '0017_000355', '0017_000367', '0017_000701', '0017_000707', '0017_000714', '0017_000720', '0017_000009', '0017_000015', '0017_000017', '0017_000019', '0017_001051', '0017_001064', '0017_001068', '0017_001070', '0017_001403', '0017_001406', '0017_001415', '0017_001419', '0018_000362', '0018_000365', '0018_000368', '0018_000369', '0018_000702', '0018_000703', '0018_000705', '0018_000710', '0018_000006', '0018_000009', '0018_000015', '0018_000019', '0018_001053', '0018_001056', '0018_001061', '0018_001064', '0018_001401', '0018_001405', '0018_001411', '0018_001418', '0019_000355', '0019_000357', '0019_000362', '0019_000365', '0019_000701', '0019_000702', '0019_000706', '0019_000712', '0019_000007', '0019_000010', '0019_000016', '0019_000020', '0019_001053', '0019_001056', '0019_001064', '0019_001068', '0019_001403', '0019_001408', '0019_001414', '0019_001420', '0020_000352', '0020_000355', '0020_000360', '0020_000370', '0020_000705', '0020_000711', '0020_000715', '0020_000719', '0020_000002', '0020_000006', '0020_000011', '0020_000018', '0020_001054', '0020_001060', '0020_001066', '0020_001070', '0020_001409', '0020_001410', '0020_001416', '0020_001419']
    control_filenames = ['0011_000354', '0011_000357', '0011_000361', '0011_000367', '0011_000701', '0011_000705', '0011_000710', '0011_000716', '0011_000002', '0011_000003', '0011_000014', '0011_000018', '0011_001054', '0011_001057', '0011_001064', '0011_001069', '0011_001402', '0011_001403', '0011_001411', '0011_001412', '0012_000353', '0012_000356', '0012_000362', '0012_000368', '0012_000702', '0012_000703', '0012_000706', '0012_000720', '0012_000007', '0012_000012', '0012_000016', '0012_000018', '0012_001054', '0012_001056', '0012_001057', '0012_001058', '0012_001404', '0012_001406', '0012_001408', '0012_001419', '0013_000354', '0013_000356', '0013_000357', '0013_000360', '0013_000708', '0013_000710', '0013_000715', '0013_000720', '0013_000002', '0013_000009', '0013_000012', '0013_000017', '0013_001052', '0013_001053', '0013_001056', '0013_001067', '0013_001404', '0013_001410', '0013_001415', '0013_001420', '0014_000354', '0014_000367', '0014_000368', '0014_000369', '0014_000703', '0014_000704', '0014_000714', '0014_000715', '0014_000001', '0014_000004', '0014_000005', '0014_000020', '0014_001051', '0014_001058', '0014_001062', '0014_001067', '0014_001402', '0014_001411', '0014_001414', '0014_001419', '0015_000358', '0015_000361', '0015_000364', '0015_000370', '0015_000702', '0015_000704', '0015_000708', '0015_000711', '0015_000001', '0015_000002', '0015_000003', '0015_000011', '0015_001059', '0015_001061', '0015_001064', '0015_001068', '0015_001408', '0015_001411', '0015_001416', '0015_001419', '0016_000360', '0016_000362', '0016_000364', '0016_000367', '0016_000701', '0016_000708', '0016_000712', '0016_000717', '0016_000005', '0016_000010', '0016_000011', '0016_000020', '0016_001058', '0016_001062', '0016_001067', '0016_001069', '0016_001401', '0016_001411', '0016_001412', '0016_001416', '0017_000351', '0017_000354', '0017_000355', '0017_000367', '0017_000701', '0017_000707', '0017_000714', '0017_000720', '0017_000009', '0017_000015', '0017_000017', '0017_000019', '0017_001051', '0017_001064', '0017_001068', '0017_001070', '0017_001403', '0017_001406', '0017_001415', '0017_001419', '0018_000362', '0018_000365', '0018_000368', '0018_000369', '0018_000702', '0018_000703', '0018_000705', '0018_000710', '0018_000006', '0018_000009', '0018_000015', '0018_000019', '0018_001053', '0018_001056', '0018_001061', '0018_001064', '0018_001401', '0018_001405', '0018_001411', '0018_001418', '0019_000355', '0019_000357', '0019_000362', '0019_000365', '0019_000701', '0019_000702', '0019_000706', '0019_000712', '0019_000007', '0019_000010', '0019_000016', '0019_000020', '0019_001053', '0019_001056', '0019_001064', '0019_001068', '0019_001403', '0019_001408', '0019_001414', '0019_001420', '0020_000352', '0020_000355', '0020_000360', '0020_000370', '0020_000705', '0020_000711', '0020_000715', '0020_000719', '0020_000002', '0020_000006', '0020_000011', '0020_000018', '0020_001054', '0020_001060', '0020_001066', '0020_001070', '0020_001409', '0020_001410', '0020_001416', '0020_001419']
elif dataset=="LibriTTSR":
    pass
    # results = {
    #     # "Ground-Truth": f'/mntcephfs/lab_data/shoinoue/Models/trained_models/ATSIP/FastSpeech2/evaluation/LibriTTSR/audio/ground-truth/',
    #     # "External": f'/mntcephfs/lab_data/shoinoue/Models/trained_models/ATSIP/FastSpeech2/evaluation/LibriTTSR/audio/External/',
    #     "VA": f'/mntcephfs/lab_data/shoinoue/Models/trained_models/ATSIP/FastSpeech2/evaluation/LibriTTSR/audio/Variance-Adaptor/',
    #     # "VA-seq": f'/mntcephfs/lab_data/shoinoue/Models/trained_models/ATSIP/FastSpeech2/evaluation/LibriTTSR/audio/Variance-Adaptor_seq/',
    # }
    # depth = 3
    # mcd_filenames = ['1272_128104_000003_000005', '1272_128104_000005_000000', '1272_135031_000052_000001', '1272_135031_000058_000000', '1272_141231_000031_000000', '1462_170138_000007_000004', '1462_170138_000022_000012', '1462_170142_000024_000002', '1462_170142_000043_000006', '1462_170145_000006_000001', '1462_170145_000011_000000', '1462_170145_000031_000001', '1462_170145_000031_000002', '174_168635_000030_000001', '174_168635_000042_000002', '174_50561_000001_000008', '174_84280_000003_000003', '174_84280_000004_000003', '174_84280_000016_000000', '1919_142785_000020_000006', '1919_142785_000035_000002', '1988_147956_000002_000000', '1988_24833_000005_000000', '1988_24833_000006_000002', '1988_24833_000009_000002', '1988_24833_000019_000001', '1988_24833_000050_000001', '1988_24833_000058_000000', '1988_24833_000062_000000', '1993_147149_000022_000002', '1993_147149_000036_000000', '1993_147149_000043_000000', '1993_147964_000005_000009', '1993_147964_000008_000000', '2035_147960_000004_000009', '2035_147960_000007_000001', '2035_147961_000004_000005', '2035_147961_000007_000006', '2035_147961_000009_000004', '2035_147961_000013_000000', '2035_147961_000013_000008', '2035_147961_000015_000001', '2035_147961_000017_000001', '2035_147961_000017_000005', '2078_142845_000004_000000', '2078_142845_000044_000001', '2086_149214_000006_000004', '2086_149220_000007_000003', '2086_149220_000016_000003', '2086_149220_000027_000001', '2086_149220_000028_000000', '2086_149220_000029_000001', '2086_149220_000034_000000', '2086_149220_000047_000000', '2277_149874_000025_000003', '2277_149896_000017_000000', '2277_149897_000006_000004', '2277_149897_000009_000000', '2277_149897_000016_000001', '2277_149897_000020_000004', '2277_149897_000038_000000', '2277_149897_000040_000001', '2277_149897_000045_000000', '2412_153954_000007_000005', '2412_153954_000007_000006', '2412_153954_000008_000002', '2412_153954_000008_000003', '2428_83699_000008_000007', '2428_83699_000013_000004', '2428_83699_000039_000008', '2428_83705_000022_000000', '2428_83705_000025_000002', '2428_83705_000033_000005', '251_118436_000012_000002', '251_118436_000019_000000', '251_118436_000025_000000', '251_118436_000026_000000', '251_136532_000011_000000', '251_136532_000019_000002', '251_136532_000020_000000', '251_136532_000020_000001', '251_137823_000013_000000', '251_137823_000022_000001', '251_137823_000024_000003', '251_137823_000033_000002', '251_137823_000042_000001', '251_137823_000044_000001', '251_137823_000052_000000', '2803_154320_000004_000004', '2803_154328_000007_000000', '2803_154328_000039_000000', '2803_154328_000047_000000', '2803_154328_000065_000000', '2803_154328_000069_000000', '2803_161169_000005_000008', '2902_9008_000002_000000', '3000_15664_000002_000000', '3000_15664_000004_000001', '3000_15664_000014_000000', '3000_15664_000039_000003', '3170_137482_000008_000000', '3170_137482_000016_000001', '3170_137482_000022_000000', '3170_137482_000032_000000', '3170_137482_000058_000008', '3536_23268_000008_000000', '3536_8226_000020_000006', '3536_8226_000027_000002', '3752_4943_000054_000004', '3752_4943_000059_000003', '3752_4944_000029_000000', '3752_4944_000033_000000', '3752_4944_000055_000001', '3752_4944_000059_000000', '3752_4944_000100_000007', '3853_163249_000035_000000', '3853_163249_000075_000001', '3853_163249_000117_000000', '3853_163249_000146_000001', '5338_284437_000041_000001', '5536_43359_000001_000001', '5536_43359_000017_000004', '5536_43359_000020_000000', '5536_43363_000017_000000', '5694_64025_000018_000000', '5694_64025_000019_000002', '5694_64029_000008_000001', '5694_64029_000012_000001', '5694_64029_000019_000000', '5694_64038_000015_000003', '5895_34615_000019_000006', '5895_34615_000023_000000', '5895_34622_000005_000003', '5895_34622_000038_000000', '5895_34629_000006_000001', '5895_34629_000036_000000', '6241_61943_000009_000000', '6241_61943_000023_000000', '6241_61946_000033_000001', '6241_66616_000012_000001', '6295_244435_000015_000001', '6295_64301_000002_000001', '6313_66125_000016_000000', '6313_66125_000064_000001', '6313_66129_000016_000001', '6313_66129_000036_000001', '6313_66129_000038_000000', '6313_66129_000064_000002', '6313_66129_000088_000000', '6313_76958_000024_000000', '6313_76958_000025_000000', '6313_76958_000038_000001', '6313_76958_000048_000000', '6313_76958_000083_000000', '6313_76958_000089_000000', '6313_76958_000094_000000', '6319_275224_000015_000000', '6319_275224_000018_000000', '6319_275224_000021_000002', '6319_275224_000022_000001', '6319_275224_000022_000007', '6319_57405_000007_000001', '6319_64726_000015_000001', '6345_64257_000008_000008', '6345_64257_000009_000011', '6345_93302_000036_000002', '6345_93302_000037_000007', '6345_93302_000062_000003', '6345_93306_000023_000003', '6345_93306_000043_000001', '652_130737_000021_000000', '777_126732_000076_000005', '7850_286674_000002_000004', '7850_286674_000005_000000', '7850_73752_000003_000015', '7850_73752_000009_000004', '7976_105575_000011_000010', '7976_105575_000014_000000', '7976_105575_000014_000004', '7976_105575_000020_000002', '7976_110124_000026_000000', '7976_110124_000031_000001', '7976_110124_000063_000000', '7976_110523_000006_000000', '7976_110523_000010_000001', '7976_110523_000027_000002', '8297_275154_000000_000001', '8297_275154_000008_000002', '8297_275155_000028_000006', '8297_275155_000029_000000', '8297_275155_000038_000001', '8297_275156_000013_000006', '8297_275156_000015_000001', '8297_275156_000017_000002', '84_121123_000009_000008', '84_121123_000011_000007', '84_121123_000028_000000', '84_121550_000010_000000', '84_121550_000097_000000', '84_121550_000193_000000']
    # control_filenames = ['1272_128104_000003_000005', '1272_128104_000005_000000', '1272_135031_000052_000001', '1272_135031_000058_000000', '1272_141231_000031_000000', '1462_170138_000007_000004', '1462_170138_000022_000012', '1462_170142_000024_000002', '1462_170142_000043_000006', '1462_170145_000006_000001', '1462_170145_000011_000000', '1462_170145_000031_000001', '1462_170145_000031_000002', '174_168635_000030_000001', '174_168635_000042_000002', '174_50561_000001_000008', '174_84280_000003_000003', '174_84280_000004_000003', '174_84280_000016_000000', '1919_142785_000020_000006', '1919_142785_000035_000002', '1988_147956_000002_000000', '1988_24833_000005_000000', '1988_24833_000006_000002', '1988_24833_000009_000002', '1988_24833_000019_000001', '1988_24833_000050_000001', '1988_24833_000058_000000', '1988_24833_000062_000000', '1993_147149_000022_000002', '1993_147149_000036_000000', '1993_147149_000043_000000', '1993_147964_000005_000009', '1993_147964_000008_000000', '2035_147960_000004_000009', '2035_147960_000007_000001', '2035_147961_000004_000005', '2035_147961_000007_000006', '2035_147961_000009_000004', '2035_147961_000013_000000', '2035_147961_000013_000008', '2035_147961_000015_000001', '2035_147961_000017_000001', '2035_147961_000017_000005', '2078_142845_000004_000000', '2078_142845_000044_000001', '2086_149214_000006_000004', '2086_149220_000007_000003', '2086_149220_000016_000003', '2086_149220_000027_000001', '2086_149220_000028_000000', '2086_149220_000029_000001', '2086_149220_000034_000000', '2086_149220_000047_000000', '2277_149874_000025_000003', '2277_149896_000017_000000', '2277_149897_000006_000004', '2277_149897_000009_000000', '2277_149897_000016_000001', '2277_149897_000020_000004', '2277_149897_000038_000000', '2277_149897_000040_000001', '2277_149897_000045_000000', '2412_153954_000007_000005', '2412_153954_000007_000006', '2412_153954_000008_000002', '2412_153954_000008_000003', '2428_83699_000008_000007', '2428_83699_000013_000004', '2428_83699_000039_000008', '2428_83705_000022_000000', '2428_83705_000025_000002', '2428_83705_000033_000005', '251_118436_000012_000002', '251_118436_000019_000000', '251_118436_000025_000000', '251_118436_000026_000000', '251_136532_000011_000000', '251_136532_000019_000002', '251_136532_000020_000000', '251_136532_000020_000001', '251_137823_000013_000000', '251_137823_000022_000001', '251_137823_000024_000003', '251_137823_000033_000002', '251_137823_000042_000001', '251_137823_000044_000001', '251_137823_000052_000000', '2803_154320_000004_000004', '2803_154328_000007_000000', '2803_154328_000039_000000', '2803_154328_000047_000000', '2803_154328_000065_000000', '2803_154328_000069_000000', '2803_161169_000005_000008', '2902_9008_000002_000000', '3000_15664_000002_000000', '3000_15664_000004_000001', '3000_15664_000014_000000', '3000_15664_000039_000003', '3170_137482_000008_000000', '3170_137482_000016_000001', '3170_137482_000022_000000', '3170_137482_000032_000000', '3170_137482_000058_000008', '3536_23268_000008_000000', '3536_8226_000020_000006', '3536_8226_000027_000002', '3752_4943_000054_000004', '3752_4943_000059_000003', '3752_4944_000029_000000', '3752_4944_000033_000000', '3752_4944_000055_000001', '3752_4944_000059_000000', '3752_4944_000100_000007', '3853_163249_000035_000000', '3853_163249_000075_000001', '3853_163249_000117_000000', '3853_163249_000146_000001', '5338_284437_000041_000001', '5536_43359_000001_000001', '5536_43359_000017_000004', '5536_43359_000020_000000', '5536_43363_000017_000000', '5694_64025_000018_000000', '5694_64025_000019_000002', '5694_64029_000008_000001', '5694_64029_000012_000001', '5694_64029_000019_000000', '5694_64038_000015_000003', '5895_34615_000019_000006', '5895_34615_000023_000000', '5895_34622_000005_000003', '5895_34622_000038_000000', '5895_34629_000006_000001', '5895_34629_000036_000000', '6241_61943_000009_000000', '6241_61943_000023_000000', '6241_61946_000033_000001', '6241_66616_000012_000001', '6295_244435_000015_000001', '6295_64301_000002_000001', '6313_66125_000016_000000', '6313_66125_000064_000001', '6313_66129_000016_000001', '6313_66129_000036_000001', '6313_66129_000038_000000', '6313_66129_000064_000002', '6313_66129_000088_000000', '6313_76958_000024_000000', '6313_76958_000025_000000', '6313_76958_000038_000001', '6313_76958_000048_000000', '6313_76958_000083_000000', '6313_76958_000089_000000', '6313_76958_000094_000000', '6319_275224_000015_000000', '6319_275224_000018_000000', '6319_275224_000021_000002', '6319_275224_000022_000001', '6319_275224_000022_000007', '6319_57405_000007_000001', '6319_64726_000015_000001', '6345_64257_000008_000008', '6345_64257_000009_000011', '6345_93302_000036_000002', '6345_93302_000037_000007', '6345_93302_000062_000003', '6345_93306_000023_000003', '6345_93306_000043_000001', '652_130737_000021_000000', '777_126732_000076_000005', '7850_286674_000002_000004', '7850_286674_000005_000000', '7850_73752_000003_000015', '7850_73752_000009_000004', '7976_105575_000011_000010', '7976_105575_000014_000000', '7976_105575_000014_000004', '7976_105575_000020_000002', '7976_110124_000026_000000', '7976_110124_000031_000001', '7976_110124_000063_000000', '7976_110523_000006_000000', '7976_110523_000010_000001', '7976_110523_000027_000002', '8297_275154_000000_000001', '8297_275154_000008_000002', '8297_275155_000028_000006', '8297_275155_000029_000000', '8297_275155_000038_000001', '8297_275156_000013_000006', '8297_275156_000015_000001', '8297_275156_000017_000002', '84_121123_000009_000008', '84_121123_000011_000007', '84_121123_000028_000000', '84_121550_000010_000000', '84_121550_000097_000000', '84_121550_000193_000000']
    # text_base_dir = "/mntcephfs/lee_dataset/tts/libritts/LibriTTS/"
    
def getfiles(txtfile):
    # Read raw bytes
    with open(txtfile, 'rb') as f:
        raw = f.read()
    # Let chardet guess
    guess = chardet.detect(raw)
    enc = guess['encoding']
    confidence = guess['confidence']
    # print(f"Detected {enc} with {confidence*100:.1f}% confidence")
    # Re-open with that encoding
    with open(txtfile, 'r', encoding=enc, errors='replace') as f:
        a = f.readlines()
    return a

gttext_dir = {}
txtlist = glob.glob("../../Dataset/ESD/*/*.txt")
txtlist.sort()
for txtfile in txtlist:
    print(txtfile)
    l = getfiles(txtfile)
    for line in l:
        a = line.split("\t")
        if len(a)==3:
            gttext_dir[a[0]] = normalizer(a[1])

# WER

In [ ]:
reset_result = False

result_dir = result_base_dir + f"{dataset}/wer/"
def init_scores():
    indv_scores = {}
    axis = "linear"
    indv_scores[axis] = {}
    for f in features:
        indv_scores[axis][f] = []
    return indv_scores

scores = {}
available_models = list(results.keys())
for p, m in enumerate(available_models):
    name = m
    print(name)

    if reset_result:
        notfound = True
    else:
        try:
            a = np.load(result_dir + f"{name}.npy", allow_pickle=True)
            print("LOAD")
            scores[name] = a
            notfound = False
        except FileNotFoundError:
            notfound = True

    if notfound:
        audiofiles = glob.glob(results[m] + "*.wav")[:3]
        audiofiles.sort()

        wers = []
        cers = []
        for size in models:
            predtexts = []
            gttexts = []
            for path in tqdm(audiofiles):
                predtext = normalizer(models[size].transcribe(path)["text"])
                predtexts += [predtext]

                gttext = gttext_dir[os.path.basename(path)[:11]]
                gttexts += [gttext]

            wer = jiwer.wer(gttexts, predtexts)
            cer = jiwer.cer(gttexts, predtexts)
            wers += [wer]
            cers += [cer]

        os.makedirs(result_dir, exist_ok=True)
        np.save(result_dir + f"{name}.npy", [wers, cers])
        print("SAVE")
        scores[name] = np.array([wers, cers])

In [ ]:
pd.DataFrame(np.array(list(scores.values()))[:,:,0], columns=["WER", "CER"], index=scores)